In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import pickle

#Code to insert into df the scraping information as dicts
#Code found on https://stackoverflow.com/questions/41825868/update-python-dictionary-add-another-value-to-existing-key/41826126#41826126
#Wrote by user https://stackoverflow.com/users/1860929/mu-%e7%84%a1
def set_key(dictionary, key, value):
     if key not in dictionary:
         dictionary[key] = value
     elif type(dictionary[key]) == list:
         dictionary[key].append(value)
     else:
         dictionary[key] = [dictionary[key], value]

In [2]:
main_url = 'https://www.transfermarkt.us'
europe_url = 'https://www.transfermarkt.us/wettbewerbe/europa'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
seconds = 0.2

# Run this only if you haven't scraped any data yet

In [114]:
#Final Dataframe with player informations
df = pd.DataFrame()

#Creates soup and request object
response_league = requests.get(europe_url, time.sleep(seconds), headers = headers)

soup_league = BeautifulSoup(response_league.content, 'html.parser', from_encoding="iso-8859-1")

#Lists that we'll store data we still need to scrape and data we already scraped
league_links = []
league_links_done = []

team_links = []
team_links_done = []


player_links = []
player_links_done = []

#Gets league link
for league in soup_league.find('tbody').findAll(href = True, title = True):
    if league['href'].split('/')[2] == 'startseite':
        league_links.append(league['href'])
        

# Start running this after alrealdy scraped data

#### Do not run the cell above if it's the first time you're scraping the data, only the next one

In [3]:
#Loading df
df = pd.read_csv('scrapped_df.csv')

#Loading lists
with open("lists_file.txt", 'rb') as f:
    league_links, league_links_done, team_links, team_links_done, player_links, player_links_done = pickle.load(f)

In [4]:
#Start running this if you haven't scraped any data yet

#create copy of list so that it allows us to use remove items from the list as we iterate through it
league_links[:] = [league_tup for league_tup in league_links if league_tup not in league_links_done]

#Opens link for each league
for league_url in league_links[:]:
    #Creates league url
    league_profile_url = main_url + league_url
    
    #Gets response for url
    response_league = requests.get(league_profile_url , time.sleep(seconds), headers = headers)
    
    #Creates bs object
    soup_league = BeautifulSoup(response_league.content, 'html.parser', from_encoding="iso-8859-1")
    
    #Gets each team's url for the league and appends to the team_links list if it's the first time scraping    
    for team in soup_league.findAll('a',  href = True, attrs = {"class": "vereinprofil_tooltip"}):
        if (team['href'] not in team_links) & (team['href'].split('/')[2] == 'startseite'):
            team_links.append(team['href'])
    
    #Gets league country
    league_country = soup_league.find('div', attrs = {"class" : "flagge"}).find('img').get('title')
    
    #since there could be players from sub-20 teams, the league name get's confused, this will avoid that from happening
    league_name = soup_league.findAll('h1', attrs = {"class" : "spielername-profil"})[0].text
    
    #create copy of list so that it allows us to use remove items from the list as we iterate through it
    team_links[:] = [team_tup for team_tup in team_links if team_tup not in team_links_done]
    
    for club in team_links[:]:
        #Defines the last season year depending if the season is per year or per semester
        if '/' in soup_league.find('h2', attrs = {"class" : "table-header"}).text.split(' ')[-1]:
            this_season = int('20' + soup_league.find('h2', attrs = {"class" : "table-header"}).text.split(' ')[-1].split('/')[0])
        else:
            this_season = int(soup_league.find('h2', attrs = {"class" : "table-header"}).text.split(' ')[-1]) - 1
        
        #Gets team url for each team in the teams list
        team_profile_url = main_url + club
        
        #Gets response for url
        response_team = requests.get(team_profile_url , time.sleep(seconds), headers = headers)

        #Creates bs object
        soup_team = BeautifulSoup(response_team.content, 'html.parser', from_encoding="iso-8859-1")
        
        #some clubs do not have data but still show up in the list, this will prevent the code from stopping
        try:
            soup_team.find(attrs = {"class" : "empty"}).text
        except:
            #Get profile for each player of the team and appends it to player_links        
            for player_url in soup_team.find('table', attrs = {"class": "items"}).findAll('a',  href = True, attrs = {"class": "spielprofil_tooltip"}):
                if player_url['href'] not in player_links:
                    player_links.append(player_url['href'])


            
            #create copy of list so that it allows us to use remove items from the list as we iterate through it
            player_links[:] = [player_tup for player_tup in player_links if player_tup not in player_links_done]

            #Gets information for each player of the club
            for player in player_links[:]:

                #Adds players profile link to the url page
                player_profile_url =  main_url + player

                #Creates link to player's season stats
                player_stats_url = player_profile_url.replace('profil', 'leistungsdaten') + "/plus/0?saison=" + str(this_season)
                player_last_stats_url = player_profile_url.replace('profil', 'leistungsdaten') + "/plus/0?saison=" + str(this_season - 1)


                #Gets response for prifle url
                response_player = requests.get(player_profile_url , time.sleep(seconds), headers = headers)

                #Gets response for stats url
                response_player_stats = requests.get(player_stats_url , time.sleep(seconds), headers = headers)
                response_player_last_stats = requests.get(player_last_stats_url , time.sleep(seconds), headers = headers)


                #Creates bs object
                soup_player = BeautifulSoup(response_player.content, 'html.parser', from_encoding="iso-8859-1")

                #Creates bs object
                soup_player_stats = BeautifulSoup(response_player_stats.content, 'html.parser', from_encoding="iso-8859-1")
                soup_player_last_stats = BeautifulSoup(response_player_last_stats.content, 'html.parser', from_encoding="iso-8859-1")


                 #Creates dict that we'll store the first step of the scraping
                info = {}

                #Gets most information from the player's profile
                for z in soup_player.find('table', attrs = { "class" : "auflistung"}).findAll('tr'):
                    set_key(info, z.th.text.strip().replace(':',''),  z.td.text.strip())

                #Scraping extra information and stats
                if info['Position'] != 'Goalkeeper':

                    #Season scrapped
                    try:
                        set_key(info, 'current_season', soup_player_stats.findAll('td', attrs = {"class" : "rechts"})[0].text.split(' ')[-1].split(':')[0])
                    except:
                        pass

                    #Checks if player has season stats to scrap
                    try :
                        soup_player_stats.find(attrs = {"class" : "empty"}).text                    
                    except:

                        #Appearances
                        try:
                            set_key(info, 'appearances', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[0].text)
                        except:
                            pass

                        #Goals
                        try:
                            set_key(info, 'goals', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[1].text)
                        except:
                            pass

                        #Assists
                        try:
                            set_key(info, 'assists', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[2].text)
                        except:
                            pass

                        #Yellow cards
                        try:
                            set_key(info, 'yellow', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[3].text)
                        except:
                            pass

                        #Second Yellow cards
                        try:
                            set_key(info, 'second_yellow', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[4].text)
                        except:
                            pass

                        #Red
                        try:
                            set_key(info, 'red', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[5].text)
                        except:
                            pass

                        #Minutes played
                        try:
                            set_key(info, 'minutes_played', soup_player_stats.findAll('td', attrs = {"class" : "rechts"})[1].text)
                        except:
                            pass

                        #Last season stats -------------------------------------------------------------------------
                       #Checks if player has last season stats to scrap
                    try :
                        soup_player_last_stats.find(attrs = {"class" : "empty"}).text                    
                    except:
                            #Appearances
                        try:
                            set_key(info, 'last_appearances', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[0].text)
                        except:
                            pass

                        #Goals
                        try:
                            set_key(info, 'last_goals', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[1].text)
                        except:
                            pass

                        #Assists
                        try:
                            set_key(info, 'last_assists', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[2].text)
                        except:
                            pass

                        #Yellow cards
                        try:
                            set_key(info, 'last_yellow', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[3].text)
                        except:
                            pass

                        #Second Yellow cards
                        try:
                            set_key(info, 'last_second_yellow', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[4].text)
                        except:
                            pass

                        #Red
                        try:
                            set_key(info, 'last_red', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[5].text)
                        except:
                            pass

                        #Minutes played
                        try:
                            set_key(info, 'last_minutes_played', soup_player_last_stats.findAll('td', attrs = {"class" : "rechts"})[1].text)
                        except:
                            pass

                    #Alternative name
                    try:
                        set_key(info, 'short_name', soup_player.find('h1', attrs = { "itemprop" : "name"}).text)
                    except:
                        pass

                    #League
                    try:
                        set_key(info, 'league', league_name)
                    except:
                        pass
                    
                    #League country
                    try:
                        set_key(info, 'league_country', league_country)
                    except:
                        pass

                    #Market value
                    try:
                        set_key(info, 'latest_market_value', soup_player.findAll('div', attrs = { "class" : "right-td"})[0].text.strip().split(' ')[0])
                    except:
                        pass

                    #Highest ever market value
                    try:
                        set_key(info, 'highest_market_value', soup_player.findAll('div', attrs = { "class" : "right-td"})[2].text.strip().split(' ')[0])
                    except:
                        pass

                    #Main position, three different excepts because depending on the player, the html can be in a different side
                    try:
                        set_key(info, 'main_position', soup_player.find('div', attrs = { "class" : "hauptposition-left"}).text.strip().split(':')[1])
                    except:
                        try:
                            set_key(info, 'main_position', soup_player.find('div', attrs = { "class" : "hauptposition-right"}).text.strip().split(':')[1])
                        except:
                            try:
                                set_key(info, 'main_position', soup_player.find('div', attrs = { "class" : "hauptposition-center"}).text.strip().split(':')[1])
                            except:
                                pass

                    #Secondary position
                    try:
                        set_key(info, 'secondary_position', soup_player.find('div', attrs = { "class" : "nebenpositionen"}).text.strip().split('\n')[1])
                    except:
                        pass


                #In case the player is a keeper, extra scraping needs to be different
                else:                
                    #Season scrapped
                    try:
                        set_key(info, 'current_season', soup_player_stats.findAll('td', attrs = {"class" : "rechts"})[0].text.split(' ')[-1].split(':')[0])
                    except:
                        pass

                    #Checks if keeper has season stats to scrap
                    try:
                        soup_player_stats.find(attrs = {"class" : "empty"}).text
                    except:

                        #Appearances
                        try:
                            set_key(info, 'appearances', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[0].text)
                        except:
                            pass

                        #Goals
                        try:
                            set_key(info, 'goals', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[1].text)
                        except:
                            pass

                        #Yellow cards
                        try:
                            set_key(info, 'yellow', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[2].text)
                        except:
                            pass

                        #Second Yellow cards
                        try:
                            set_key(info, 'second_yellow', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[3].text)
                        except:
                            pass

                        #Red cards
                        try:
                            set_key(info, 'red', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[4].text)
                        except:
                            pass

                        #Goals conceeded
                        try:
                            set_key(info, 'goals_conceded', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[5].text)
                        except:
                            pass

                        #Clean sheets
                        try:
                            set_key(info, 'clean_sheets', soup_player_stats.findAll('td', attrs = {"class" : "zentriert"})[6].text)
                        except:
                            pass
                        #minutes played
                        try:
                            set_key(info, 'minutes_played', soup_player_stats.findAll('td', attrs = {"class" : "rechts"})[1].text)
                        except:
                            pass

                        #Last season stats --------------------------
                        #Appearances
                        #Checks if keeper has last season stats to scrap
                    try :
                        soup_player_last_stats.find(attrs = {"class" : "empty"}).text                    
                    except:
                        try:
                            set_key(info, 'last_appearances', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[0].text)
                        except:
                            pass

                        #Goals
                        try:
                            set_key(info, 'last_goals', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[1].text)
                        except:
                            pass

                        #Yellow cards
                        try:
                            set_key(info, 'last_yellow', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[2].text)
                        except:
                            pass

                        #Second Yellow cards
                        try:
                            set_key(info, 'last_second_yellow', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[3].text)
                        except:
                            pass

                        #Red cards
                        try:
                            set_key(info, 'last_red', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[4].text)
                        except:
                            pass

                        #Goals conceeded
                        try:
                            set_key(info, 'last_goals_conceded', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[5].text)
                        except:
                            pass

                        #Clean sheets
                        try:
                            set_key(info, 'last_clean_sheets', soup_player_last_stats.findAll('td', attrs = {"class" : "zentriert"})[6].text)
                        except:
                            pass

                        #Minutes played
                        try:
                            set_key(info, 'last_minutes_played', soup_player_last_stats.findAll('td', attrs = {"class" : "rechts"})[1].text)
                        except:
                            pass

                    #Alternative name
                    try:
                        set_key(info, 'short_name', soup_player.find('h1', attrs = { "itemprop" : "name"}).text)
                    except:
                        pass

                     #League
                    try:
                        set_key(info, 'league', league_name)
                    except:
                        pass
                    
                      #League
                    try:
                        set_key(info, 'league_country', league_country)
                    except:
                        pass

                    #Market value
                    try:
                        set_key(info, 'latest_market_value', soup_player.findAll('div', attrs = { "class" : "right-td"})[0].text.strip().split(' ')[0])
                    except:
                        pass

                    #Highest ever market value
                    try:
                        set_key(info, 'highest_market_value', soup_player.findAll('div', attrs = { "class" : "right-td"})[2].text.strip().split(' ')[0])
                    except:
                        pass

                info = pd.DataFrame.from_dict(info, orient = 'index').transpose()
                df = pd.concat([df,info])

                #Stores players already scraped
                player_links_done.append(player)                

        #Stores teams already scraped
        team_links_done.append(club)
        
    #Store leagues already scraped
    league_links_done.append(league_url)

In [5]:
#Run if the df is not done yet
df.to_csv('scraped_df.csv', index = False)

#Saving variables
with open("lists_file.txt", 'wb') as f:
    pickle.dump((league_links, league_links_done, team_links, team_links_done, player_links, player_links_done), f)